In [2]:
from NPZGraphDataset import NPZGraphDataset
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader

In [3]:
# 1) 读取 manifest
manifest_path = Path("./graph_npz/manifest.csv")
root_dir = Path("./")  # 若 file_path 是相对路径，作为公共前缀

manifest = pd.read_csv(manifest_path)

In [4]:
# 2) 分层标签：把连续的 Tg 分成若干箱做 stratify（若标签太少会自动退化为非分层）
if "label" in manifest:
    # 这里用 10 个分箱；你也可以改成 5 或 20
    try:
        bins = pd.qcut(manifest["label"], q=10, duplicates="drop")
        stratify_labels = bins.astype(str)
    except Exception:
        stratify_labels = None
else:
    stratify_labels = None

In [5]:
# 3) 先切出测试集（10%），再从剩余里切验证集（10%）
train_val_df, test_df = train_test_split(
    manifest,
    test_size=0.10,
    random_state=42,
    shuffle=True,
    stratify=stratify_labels if stratify_labels is not None else None,
)
# 对 train_val 再按 10% 切出 val => 0.9 * 0.1 = 9%（接近 80/10/10）
if stratify_labels is not None:
    train_val_bins = pd.qcut(train_val_df["label"], q=10, duplicates="drop")
    stratify_trainval = train_val_bins.astype(str)
else:
    stratify_trainval = None

train_df, val_df = train_test_split(
    train_val_df,
    test_size=0.10/0.90,   # 约 0.111..., 使得整体 ~80/10/10
    random_state=42,
    shuffle=True,
    stratify=stratify_trainval if stratify_trainval is not None else None,
)

print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

Train: 5893 | Val: 737 | Test: 737


In [6]:
# 4) 构建三个 Dataset（验证/测试共用训练集的标准化参数）
train_dataset = NPZGraphDataset(
    manifest=train_df,
    root=root_dir,
    separate_pos=True,      # (x,y,z) 放在 data.pos
    feature_cols=(0,1,2,3),
    coord_cols=(4,5,6),
    standardize_y=True,     # 仅用训练集统计均值方差
)

val_dataset = NPZGraphDataset(
    manifest=val_df,
    root=root_dir,
    separate_pos=True,
    feature_cols=(0,1,2,3),
    coord_cols=(4,5,6),
    standardize_y=True,     # 先开着，随后覆盖为 train 的均值方差
)
# 覆盖验证集 y 的标准化为“训练集统计”，防止数据泄漏
val_dataset._y_mean = train_dataset.y_mean
val_dataset._y_std  = train_dataset.y_std

test_dataset = NPZGraphDataset(
    manifest=test_df,
    root=root_dir,
    separate_pos=True,
    feature_cols=(0,1,2,3),
    coord_cols=(4,5,6),
    standardize_y=True,
)
# 覆盖测试集 y 的标准化为“训练集统计”，防止数据泄漏
test_dataset._y_mean = train_dataset.y_mean
test_dataset._y_std  = train_dataset.y_std
print(f"y_mean(train)={train_dataset.y_mean:.4f}, y_std(train)={train_dataset.y_std:.4f}")

y_mean(train)=417.1868, y_std(train)=112.5722


In [7]:
# 5) DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,  pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=32, shuffle=False, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False, pin_memory=True)

In [8]:
# 6) 简单检查一个 batch
batch = next(iter(train_loader))
print(batch)
print("x:", None if batch.x is None else batch.x.shape)
print("pos:", None if batch.pos is None else batch.pos.shape)
print("edge_index:", batch.edge_index.shape)
print("edge_attr:", batch.edge_attr.shape)
print("y:", batch.y.shape)

# ====== 评估时如需把预测还原为物理单位（K）：=====
# pred_real = pred_norm * train_dataset.y_std + train_dataset.y_mean
# y_real    = y_norm    * train_dataset.y_std + train_dataset.y_mean

DataBatch(x=[1155, 4], edge_index=[2, 2478], edge_attr=[2478, 4], y=[32], pos=[1155, 3], mol_id=[32], file_path=[32], batch=[1155], ptr=[33])
x: torch.Size([1155, 4])
pos: torch.Size([1155, 3])
edge_index: torch.Size([2, 2478])
edge_attr: torch.Size([2478, 4])
y: torch.Size([32])


In [11]:
import torch
import torch.nn as nn
from torch_geometric.loader import DataLoader
from Model.Modelv3withAtt.ConvModelV3 import GatedGCNModel

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
model = GatedGCNModel(
    layers_in_conv=3,
    channels=64,
    use_nodetype_coeffs=False,  # x 不是类型 one-hot，先关掉
    num_node_types=0,
    num_edge_types=4,           # 与 edge_attr 的含义无强耦合，这里保留默认
    use_jumping_knowledge=False,
    use_bias_for_update=True,
    use_dropout=True,
    num_convs=3,
    num_fc_layers=3,
    neighbors_aggr='add',
    dropout_p=0.1,
    num_targets=1,
    geom_K=16, # RBF探针数量
    geom_rmax=4.0, # 设置化学键的最大键长
    concat_original_edge=True, # 得到新的边特征后，是否与原始边特征拼接（是否使用pos信息）
    gem_out=32,
    heads=4, # GAT多头注意力机制的头数
).to(device)

In [14]:
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

In [ ]:
epochs = 50
for epoch in range(1, epochs+1):
    # ---- train ----
    model.train()
    total_loss = 0.0; n_tr = 0
    for batch in train_loader:
        batch = batch.to(device)
        pred = model(batch)                 # [B, 1]
        y = (batch.y.view(-1,1).float()).to(device)
        loss = criterion(pred, y)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()

        total_loss += loss.item() * batch.num_graphs
        n_tr += batch.num_graphs

    # ---- eval ----
    model.eval()
    mae = 0.0; rmse = 0.0; n_val = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = batch.to(device)
            pred = (model(batch))   
            y_true = batch.y.view(-1,1).float().to(device)
            mae  += (pred - y_true).abs().sum().item()
            rmse += ((pred - y_true)**2).sum().item()
            n_val += batch.num_graphs
    mae  /= n_val
    rmse = (rmse / n_val) ** 0.5

    print(f"Epoch {epoch:03d} | TrainLoss {total_loss/n_tr:.4f} | Val MAE {mae:.3f} | Val RMSE {rmse:.3f}")